# ISL Dataprep
## Run trimming, mediapipe pose estimation, metadata etc on downsampled videos

In [1]:
!nproc

24


## Set up data

Data Source: Lambda filesystem
  - bucket name: 921c05b3-8e02-4c0b-9623-d1de39939163
  - region: us-east-3


AWS creds needed:
  - AWS_ACCESS_KEY_ID=9P7Q2ADQR5WPGY83629B
  - AWS_SECRET_ACCESS_KEY=*********
  - AWS_REGION=us-east-3
  - S3_ENDPOINT_URL="https://files.us-east-3.lambda.ai"

Change the bucket and credentials when using a different instance. These are given as pattern reference.

In [ ]:
!pip install awscli boto3

In [3]:
! aws --version

aws-cli/1.42.39 Python/3.12.11 Linux/6.6.97+ botocore/1.40.39


In [4]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'C0INV916QMS8WB9F104E'
os.environ['AWS_SECRET_ACCESS_KEY'] = '******'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-3'

In [ ]:
! aws configure

In [6]:
!echo "[default]" > ~/.aws/config
!echo "region =us-east-3 " >> ~/.aws/config
!echo "endpoint-url =https://files.us-east-3.lambda.ai " >> ~/.aws/config
!echo "" >> ~/.aws/config


!echo "request_checksum_calculation =when_required " >> ~/.aws/config
!echo "response_checksum_validation =when_required " >> ~/.aws/config
!echo "" >> ~/.aws/config


In [7]:
cat ~/.aws/config

[default]
region =us-east-3 
endpoint-url =https://files.us-east-3.lambda.ai 

request_checksum_calculation =when_required 
response_checksum_validation =when_required 



In [8]:
!aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/ --endpoint-url https://files.us-east-3.lambda.ai

                           PRE islNTvideos/
                           PRE isl_gospel_videos/
                           PRE outputs/


In [ ]:
!aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/islNTvideos/ /content/islNTvideos --recursive --endpoint-url https://files.us-east-3.lambda.ai

In [ ]:
! df -h /

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   40G  187G  18% /


In [ ]:
ls /content/

1217.transcripts.json  islNTvideos/  mp4s/     sample_data/
edited_jsons/          jsons/        outputs/  sign-bibles-dataset/


In [ ]:
ls /content/islNTvideos/*/*/*.MP4 | wc

    855    1696   40930


In [ ]:
ls /content/islNTvideos/*/*.MP4 | wc

     17      34     744


In [ ]:
ls /content/islNTvideos/*/*/*.mp4 | wc

      1       3      55


In [ ]:
mkdir /content/islNTvideos/Jude/CH-1

In [ ]:
mv /content/islNTvideos/Jude/*.MP4 /content/islNTvideos/Jude/CH-1/

In [ ]:
mv /content/islNTvideos/Hebrews/*.MP4 /content/islNTvideos/Hebrews/CH-13/

In [ ]:
mv '/content/islNTvideos/1-Thessalonians/3 Ch/V- 11-13.mp4' '/content/islNTvideos/1-Thessalonians/3 Ch/V- 11-13.MP4'

In [ ]:
mv "/content/islNTvideos/Acts/Ch 27/Ch 27, V=-18-20.MP4" "/content/islNTvideos/Acts/Ch 27/Ch 27, V=18-20.MP4"

In [ ]:
ls /content/islNTvideos/*/*/*.MP4 | wc

    870    1730   41681


## Inspect Data

In [ ]:
! aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/islNTvideos/Galatians/CH-3/ --endpoint-url https://files.us-east-3.lambda.ai

In [100]:
!aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/islNTvideos/Hebrews/ /content/islNTvideos/  --recursive --endpoint-url https://files.us-east-3.lambda.ai --exclude "*" --include "20-21*"

download: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/islNTvideos/Hebrews/20-21.MP4 to islNTvideos/20-21.MP4


In [ ]:
from IPython.display import Video

# Replace with the actual path to one of your downloaded MP4 files
video_path = '/content/islNTvideos/22-25_within.MP4'
display(Video(video_path, embed=True, width=480)) # Assuming original width is around 960, setting to 50%

In [ ]:
video_path = '/content/islNTvideos/22-25.MP4'
display(Video(video_path, embed=True, width=480))

In [ ]:
from IPython.display import Video
video_path = '/content/outputs/1118.mp4'
display(Video(video_path, embed=True, width=480))

In [ ]:
from IPython.display import Video
video_path = '/content/outputs/1119.mp4'
display(Video(video_path, embed=True, width=480))

In [ ]:
import os
import cv2
import glob

input_dir = '/content/islNTvideos'
output_dir = '/content/video_frames'
os.makedirs(output_dir, exist_ok=True)

video_files = glob.glob(f'{input_dir}/*/*/*.MP4') + glob.glob(f'{input_dir}/*/*/*.mp4')

for video_path in video_files:
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error opening video file: {video_path}")
            continue

        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Check if the video has at least 200 frames
        if total_frames < 200:
            print(f"Video has less than 200 frames, skipping: {video_path}")
            cap.release()
            continue

        # Set the frame position to the 200th frame (0-indexed)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 199)

        ret, frame = cap.read()

        if ret:
            # Resize the frame to 128x128
            resized_frame = cv2.resize(frame, (128, 128))

            # Create the output filename
            book_chapter = video_path.replace(input_dir + '/', '').replace('.MP4', '').replace('.mp4', '')
            output_filename = f"{book_chapter.replace('/', '_')}_frame_200.jpg"
            output_path = os.path.join(output_dir, output_filename)

            # Save the frame
            cv2.imwrite(output_path, resized_frame)
            print(f"Saved frame 200 from {video_path} to {output_path}")
        else:
            print(f"Could not read frame 200 from {video_path}")

        cap.release()

    except Exception as e:
        print(f"An error occurred processing {video_path}: {e}")

print("Finished processing videos.")

In [ ]:
ls /content/video_frames/* |wc

    870    1730   51251


In [ ]:
from IPython.display import Image
Image(filename='/content/video_frames/1-Timothy_Ch- 1_v- 15-20 (Re-sign)_frame_200.jpg')

In [ ]:
Image(filename='/content/video_frames/Philemon_Ch-1_8-11_frame_200.jpg')

In [ ]:
import os
import glob
from IPython.display import display, clear_output
from ipywidgets import Button, VBox, Layout, Image
from google.colab import output

image_dir = '/content/video_frames'
image_files = glob.glob(f'{image_dir}/*.jpg')

# Group images by book name
image_groups = {}
for image_path in image_files:
    filename = os.path.basename(image_path)
    book_name = filename.split('_')[0]
    if book_name not in image_groups:
        image_groups[book_name] = []
    image_groups[book_name].append(image_path)

book_names = list(image_groups.keys())
book_names.sort() # Sort book names alphabetically
current_book_index = 0

def display_book_images(book_name):
    clear_output(wait=True)
    print(f"Book: {book_name}")
    images = image_groups[book_name]
    # Display images in a grid-like manner (adjust columns as needed)
    # Create ipywidgets.Image instances
    image_widgets = [Image(value=open(img, 'rb').read(), width=128, height=128) for img in images]
    for i in range(0, len(image_widgets), 8): # Display 8 images per row
        row_images = image_widgets[i:i+8]
        display(VBox(row_images, layout=Layout(flex_flow='row wrap')))

    # Display the button after showing images for the current book
    if current_book_index < len(book_names) - 1:
        button = Button(description="Next Book")
        button.on_click(on_button_clicked)
        display(button)
    else:
        print("Finished displaying all books.")


def on_button_clicked(b):
    global current_book_index
    current_book_index += 1
    if current_book_index < len(book_names):
        display_book_images(book_names[current_book_index])


if book_names:
    display_book_images(book_names[current_book_index])

else:
    print("No images found in the specified directory.")

## Set up code

In [ ]:
! mkdir /content/logs

In [ ]:
! mkdir /content/temp

In [ ]:
! mkdir /content/dataprep_output

In [ ]:
! git clone https://github.com/kavitharaju/sign-bibles-dataset.git

Cloning into 'sign-bibles-dataset'...
remote: Enumerating objects: 971, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 971 (delta 78), reused 81 (delta 70), pack-reused 750 (from 2)
Receiving objects: 100% (971/971), 4.77 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (512/512), done.


In [ ]:
cd /content/sign-bibles-dataset/dataprep/isl/

/content/sign-bibles-dataset/dataprep/isl


In [ ]:
! ls /content/sign-bibles-dataset/dataprep/isl

biblenlp_util.py	   pose_format_util.py
bible_text_access.py	   prep_BSB_bible.py
BSB_bible.json		   prep_ERV_bible.py
compare_bible_versions.py  __pycache__
dataset_README.md	   README.md
dataset_stats.json	   requirements-CPU-only.txt
docs			   requirements-dwpose-on_GPU.txt
duration_calculation.py    requirements-local-ffmpeg.txt
dwpose			   requirements_mp.txt
dwpose_processing.py	   requirements.txt
edit_data_files.py	   run_parallel_cpu.sh
ERV_bible.json		   run_parallel_gpu.sh
ffmpeg_downsample.py	   run_parallel_mp.sh
isl_bible_processing.py    s3_connect.py
isl_dict_processing.py	   test_lambda.yaml
make_tar_files.py	   test_webdataset.py
mediapipe_trim.py	   video_id_list.py
mp_processing.py	   WEB_bible.json
nextcloud_connect.py


In [ ]:
!git checkout -b ISL-NT-corrections

Switched to a new branch 'ISL-NT-corrections'


In [ ]:
!apt-get update && apt-get install -y parallel

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,526 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 ht

In [ ]:
!parallel --version

GNU parallel 20210822
Copyright (C) 2007-2021 Ole Tange, http://ole.tange.dk and Free Software
Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
GNU parallel comes with no warranty.

Web site: https://www.gnu.org/software/parallel

When using programs that use GNU Parallel to process data for publication
please cite as described in the manpage.


In [ ]:
!git pull origin ISL-NT-corrections

remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 11 (delta 7), reused 11 (delta 7), pack-reused 0 (from 0)
Unpacking objects: 100% (11/11), 972 bytes | 324.00 KiB/s, done.
From https://github.com/kavitharaju/sign-bibles-dataset
 * branch            ISL-NT-corrections -> FETCH_HEAD
 * [new branch]      ISL-NT-corrections -> origin/ISL-NT-corrections
Updating 6dc6438..d13951b
Fast-forward
 dataprep/isl/isl_bible_processing.py |  4 ++--
 dataprep/isl/mediapipe_trim.py       | 17 +++++------------
 2 files changed, 7 insertions(+), 14 deletions(-)


In [ ]:
! pip install -r requirements-CPU-only.txt

In [ ]:
! git log

## Run data processing

In [ ]:
! python video_id_list.py /content/islNTvideos/ 1114 > /content/input_list_full.txt

In [ ]:
!sed -n '1,10p' /content/input_list_full.txt > /content/input_list.txt

In [ ]:
! tail /content/input_list.txt

In [ ]:
! cat /content/input_list.txt | wc

In [ ]:
! time(bash run_parallel_cpu.sh)

In [ ]:
cat /content/logs/success.log

## Upload outputs

In [ ]:
pwd

'/content/sign-bibles-dataset/dataprep/isl'

In [ ]:
! ls /content/dataprep_output | wc

   2040    2040   33660


In [ ]:
! aws s3 cp /content/dataprep_output/ s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --recursive --endpoint-url=https://files.us-east-3.lambda.ai  #--exclude "*" --include "*.npz"

In [ ]:
! aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai | wc

   3480   13920  165300


In [ ]:
! rm  /content/dataprep_output/*

## Post dataprep analysis

### Find duration

In [ ]:
! aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/jsons/  --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.json" --exclude "*transcript*"

In [ ]:
rm /content/jsons/*

In [ ]:
ls /content/jsons/ | wc

   1740    1740   27840


In [ ]:
! python duration_calculation.py /content/jsons/

args.directories=['/content/jsons/']
Total number of videos : 870
total_seconds=69927.98000000001
Total duration: 19 hours, 25 minutes, 27 seconds
Total number of unique verses: 3201


### Check bible reference list

In [ ]:
! aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/jsons/  --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.json" --exclude "*transcript*"

In [ ]:
# list all samples in the order of vref for inspection(for missing refs, duplicates etc)

import json
from pathlib import Path

verses = []
for json_file in Path("/content/jsons/").glob("*.json"):
    with open(json_file, "r") as f:
        data = json.load(f)
        ref = data["bible-ref"]
        vrefs = data["biblenlp-vref"]
        id = json_file.name.split("/")[-1].split(".")[0]
        for vref in vrefs:
            verses.append((ref,vref, id))
verses = sorted(verses, key=lambda x: x[1])
for item in verses:
  print(f"{item[1]}   {item[0]}   {item[2]}")



### Correct refs, vrefs and bible text in jsons

In [ ]:
! aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/jsons/  --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.json" #--exclude "*transcript*"

In [ ]:
! mkdir /content/edited_jsons

In [ ]:
# Fix the ref, vref and bible text in Jude video metadata
import json
from pathlib import Path

from bible_text_access import get_verses, book_code_lookup
from biblenlp_util import ref2vref

correction_map = {
    "JUD 1:1": "JUD 1:1-2",
    "JUD 1:2": "JUD 1:3",
    "JUD 1:3": "JUD 1:4",
    "JUD 1:4": "JUD 1:5-6",
    "JUD 1:5": "JUD 1:7-8",
    "JUD 1:6": "JUD 1:9-10",
    "JUD 1:7": "JUD 1:11",
    "JUD 1:8": "JUD 1:12",
    "JUD 1:9": "JUD 1:13",
    "JUD 1:10": "JUD 1:14-15",
    "JUD 1:11": "JUD 1:17-19",
    "JUD 1:12": "JUD 1:20-23",
    "JUD 1:13": "JUD 1:24-25"

}

verses = []
for json_file in Path("/content/jsons/").glob("*.json"):
    if "transcripts" in str(json_file):
      continue
    changed = False
    data={}
    trans_data={}
    trans_file=None
    with open(json_file, "r") as f:
        data = json.load(f)
        ref = data["bible-ref"]
        if ref in correction_map:
          new_ref = correction_map[ref]
          new_vref = ref2vref(new_ref)
          data["bible-ref"] = new_ref
          data["biblenlp-vref"] = new_vref
          trans_file = str(json_file).replace(".json", ".transcripts.json")
          print(f"{trans_file=}")
          with open(trans_file, "r") as f2:
            trans_data = json.load(f2)
            trans_data[0]["bible-ref"] = new_ref
            trans_data[0]["biblenlp-vref"] = new_vref
            trans_data[0]["text"] = get_verses(new_ref, "BSB")
          changed = True
          print(f"{ref} to {new_ref}")
          # print(f"{new_vref=}")
          # print(f"{trans_data[0]['text']=}")
    if changed:
      with open(str(json_file).replace("jsons/","edited_jsons/"), "w") as f:
        json.dump(data, f, indent=4)
      with open(trans_file.replace("jsons/","edited_jsons/"), "w") as f:
        json.dump(trans_data, f, indent=4)
      print(f"{json_file} changed")





In [ ]:
! ls /content/edited_jsons/

1215.json	       1220.json	      1225.json
1215.transcripts.json  1220.transcripts.json  1225.transcripts.json
1216.json	       1221.json	      1226.json
1216.transcripts.json  1221.transcripts.json  1226.transcripts.json
1217.json	       1222.json	      1227.json
1217.transcripts.json  1222.transcripts.json  1227.transcripts.json
1218.json	       1223.json	      1228.json
1218.transcripts.json  1223.transcripts.json  1228.transcripts.json
1219.json	       1224.json
1219.transcripts.json  1224.transcripts.json


In [ ]:
import json
from pathlib import Path

from bible_text_access import get_verses, book_code_lookup
from biblenlp_util import ref2vref

json_file="/content/edited_jsons/1222.json"
data={}
trans_data={}
trans_file=None
with open(json_file, "r") as f:
    data = json.load(f)
    new_ref = "JUD 1:16"
    new_vref = ref2vref(new_ref)
    data["bible-ref"] = new_ref
    data["biblenlp-vref"] = new_vref
    trans_file = json_file.replace(".json", ".transcripts.json")
    with open(trans_file, "r") as f2:
      trans_data = json.load(f2)
      trans_data[0]["bible-ref"] = new_ref
      trans_data[0]["biblenlp-vref"] = new_vref
      trans_data[0]["text"] = get_verses(new_ref, "BSB")
    # print(f"{new_vref=}")
    # print(f"{trans_data[0]['text']=}")
with open(json_file, "w") as f:
  json.dump(data, f, indent=4)
with open(trans_file, "w") as f:
  json.dump(trans_data, f, indent=4)
print(f"{json_file} changed")



/content/edited_jsons/1222.json changed


In [ ]:
! aws s3 sync /content/edited_jsons/ s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai

In [ ]:
!aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/mp4s/ --recursive --endpoint-url https://files.us-east-3.lambda.ai --exclude "*" --include "*.mp4"

### Convert mp4 encoding

In [ ]:
! aws s3 cp s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ /content/outputs/  --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.mp4"

In [10]:
! ls /content/outputs/* | wc

    870     870   22620


In [11]:
! du /content/outputs/ -hs

47G	/content/outputs/


In [15]:
!wget https://raw.githubusercontent.com/kavitharaju/sign-bibles-dataset/refs/heads/ISL-NT-corrections/dataprep/isl/change_mp4_encoding.py

--2025-09-26 04:52:34--  https://raw.githubusercontent.com/kavitharaju/sign-bibles-dataset/refs/heads/ISL-NT-corrections/dataprep/isl/change_mp4_encoding.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 976 [text/plain]
Saving to: ‘change_mp4_encoding.py.1’

change_mp4_encoding 100%[===================>]     976  --.-KB/s    in 0s      

2025-09-26 04:52:34 (122 MB/s) - ‘change_mp4_encoding.py.1’ saved [976/976]



In [13]:
!pip install ffmpeg-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 13.9 MB/s eta 0:00:00


In [ ]:
!time(python change_mp4_encoding.py /content/outputs/)

In [17]:
! du /content/outputs/ -hs

17G	/content/outputs/


In [ ]:
! aws s3 cp  /content/outputs/ s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/  --recursive --endpoint-url=https://files.us-east-3.lambda.ai  --exclude "*" --include "*.mp4"

### Remove select samples
(duplicates and overlaps)

In [126]:
! aws s3 rm s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --recursive --endpoint-url=https://files.us-east-3.lambda.ai --exclude "*" --include "1896*"

delete: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1896.mp4
delete: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1896.json
delete: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1896.pose-dwpose.npz
delete: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1896.pose-mediapipe.npz
delete: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1896.pose-mediapipe.pose
delete: s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/1896.transcripts.json


In [128]:
! aws s3 ls s3://6a8e9853-8a1d-4662-9fa6-be70cced7c34/outputs/ --endpoint-url=https://files.us-east-3.lambda.ai  | wc

   5136   20544  254232


### Next heading